IMPORT ET CHARGEMENT DES DONNÉES NETTOYÉES

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)

In [2]:
# Chargement des données nettoyées
df = pd.read_csv("../data/processed/online_retail_clean.csv")
df.head(6)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Customer_ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom,39.6


KPI GLOBAUX

In [3]:
# Calcul des métriques globales du business
total_revenue = df['Revenue'].sum()           # Chiffre d'affaires total
total_customers = df['Customer_ID'].nunique() # Nombre de clients uniques
total_orders = df['InvoiceNo'].nunique()      # Nombre de commandes uniques
total_products = df['StockCode'].nunique()    # Nombre de produits différents vendus

total_revenue, total_customers, total_orders, total_products

(np.float64(8798233.743999999), 4312, 19213, 4017)

EVOLUTION DU CHiFFRE DANS LE TEMPS

In [4]:
# Conversion de la date en format datetime et création d'une colonne mois-année
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M').astype(str)

# Agrégation du chiffre d'affaires par mois
Monthly_revenue = (
    df.groupby('YearMonth')['Revenue']
    .sum()
    .reset_index()
)

In [5]:
# Visualisation de l'évolution du chiffre d'affaires mensuel avec Plotly
fig = px.line(
    Monthly_revenue,    # DataFrame contenant les données agrégées
    x='YearMonth',      # Axe X : périodes mensuelles
    y='Revenue',        # Axe Y : chiffre d'affaires
    title='Monthly Revenue Trend',  # Titre du graphique
    markers=True        # Affiche un point pour chaque mois
)

# Personnalisation de la mise en page
fig.update_layout(
    xaxis_title = "Month",          # Libellé axe X
    yaxis_title = "Revenue",        # Libellé axe Y
    template = "plotly_white"       # Thème visuel épuré (fond blanc)
)

fig.write_image("../reports/figures/monthly_revenue_Trend.png")
# Affichage interactif du graphique
fig.show()

CHIFFRE D'AFFAIRES PAR PAYS (PRIORISATION MARCHÉS)

In [6]:
# Agrégation du chiffre d'affaires par pays, trié du plus élevé au plus faible
country_revenue = (
    df.groupby('Country')['Revenue']  # Groupement par pays
    .sum()                            # Somme des revenus pour chaque pays
    .sort_values(ascending=False)     # Tri décroissant (meilleurs pays en premier)
    .reset_index()                    # Réinitialise l'index pour obtenir un DataFrame propre
)

In [7]:
# Création d'un graphique en barres des 10 pays les plus rentables
fig = px.bar(
    country_revenue.head(10),    # Sélection des 10 premiers pays (CA le plus élevé)
    x='Country',                 # Axe X : noms des pays
    y='Revenue',                 # Axe Y : chiffre d'affaires
    title='Top 10 Countries by Revenue',  # Titre du graphique
    text_auto='.2s'              # Affiche les valeurs sur les barres (format abrégé : 1.2k, 1.2M...)
)

# Personnalisation de la mise en page
fig.update_layout(
    template="plotly_white",              # Thème visuel épuré avec fond blanc
    yaxis={'categoryorder':'total ascending'}  # Trie les barres par ordre croissant de valeur
)

# Affichage interactif du graphique
fig.write_image("../reports/figures/Countries_revenue.png")
fig.show()

CLIENTS A HAUTE VALEUR (FONDATION DU RFM)

In [8]:
# Agrégation du chiffre d'affaires par client, trié du plus élevé au plus faible
customer_revenue = (
    df.groupby('Customer_ID')['Revenue']  # Groupement par identifiant client
    .sum()                                # Somme des revenus pour chaque client
    .sort_values(ascending=False)         # Tri décroissant (meilleurs clients en premier)
    .reset_index()                        # Réinitialise l'index pour obtenir un DataFrame propre
)

In [9]:
# Création d'un histogramme de distribution des revenus clients
fig = px.histogram(
    customer_revenue,           # DataFrame contenant le CA par client
    x='Revenue',                # Variable à distribuer sur l'axe X
    nbins=50,                   # Nombre de barres (intervalles) pour la distribution
    title='Customer Revenue Distribution'  # Titre du graphique
)

# Application du thème visuel épuré
fig.update_layout(template="plotly_white")

fig.write_image("../reports/figures/Distribution_revenue.png")
# Affichage interactif du graphique
fig.show()

INSIGHT STRATÉGIQUE 

## Principaux enseignements commerciaux

- Le chiffre d'affaires est fortement concentré sur un petit nombre de clients.

- Le Royaume-Uni domine les ventes totales, ce qui indique une forte dépendance à un seul marché.

- Des variations saisonnières marquées suggèrent des opportunités pour des campagnes ciblées.

- Un petit nombre de produits génère la majeure partie du chiffre d'affaires.